# 🔁 ARMA-CNN-BiLSTM Hybrid Model with PSO Optimization

This notebook implements a **hybrid time series forecasting model** combining traditional statistical and deep learning techniques:
- **ARMA**: captures linear dependencies in the time series
- **CNN-BiLSTM**: extracts short-term patterns and long-term temporal features from stock prices
- **PSO (Particle Swarm Optimization)**: used to optimally blend the outputs of ARMA and CNN-BiLSTM models post-training

---

## 🧱 Workflow Overview

### 1. 📥 Data Acquisition
- Dataset: **Shanghai Stock Index (000001.SS)** from Yahoo Finance
- Period: `2010-01-04` to `2020-01-23`

### 2. 🧹 Preprocessing Steps
- Outlier detection and removal using **Z-score**
- Feature scaling via **Min-Max normalization**
- Look-back window of **5 days** for time series modeling
- Dataset split: 70% Train / 10% Validation / 20% Test

### 3. ⚙️ Hybrid Model Components
- **ARMA (2,1,1)** model fitted on raw `Adi Close` prices
- **CNN-BiLSTM** deep learning model trained on scaled input features:
  - Three stacked **Conv1D** layers
  - Two **Bidirectional LSTM** layers
  - **Dropout** and **Dense** layers for regularization and output

### 4. ⚖️ PSO-Based Weight Optimization
- PSO is employed to **combine ARMA and CNN-BiLSTM predictions**
- Optimization minimizes the joint **RMSE** across train, validation, and test sets

### 5. 📊 Evaluation Metrics
Performance assessed using:
- **RMSE** (Root Mean Squared Error)
- **MAE** (Mean Absolute Error)
- **MAPE** (Mean Absolute Percentage Error)
- **R²** (Coefficient of Determination)

---

## 🔬 Paper Context

This notebook supports the **hybrid modeling section** of the research and contributes to **Table 2** in the article:

**"The Application and Effectiveness of Machine Learning and Deep Learning Methods in Analyzing and Predicting the Shanghai Stock Index"**

---

## ✅ Key Contributions
- Combines **linear modeling** (ARMA) with **deep learning** (CNN-BiLSTM)
- **PSO algorithm** improves ensemble accuracy
- Achieves **Test R² ≈ 0.95** and **MAPE < 1.3%**


In [1]:
!pip install pyswarm

  Preparing metadata (setup.py) ... done
  Created wheel for pyswarm: filename=pyswarm-0.6-py3-none-any.whl size=4463 sha256=7107ae9fa032c2c6e760dd8e4537b855e7d2bcf09fd8f5d5e9cec35dda6b4f59
  Stored in directory: /root/.cache/pip/wheels/bb/4f/ec/8970b83323e16aa95034da175454843947376614d6d5e9627f
Successfully built pyswarm


In [2]:
!pip install deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 10.8 MB/s eta 0:00:00


In [3]:
!pip install tensorflow

In [4]:
!pip install keras

In [5]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 9.4 MB/s eta 0:00:00


In [6]:
pip install --upgrade mplfinance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.1 MB/s eta 0:00:00


In [7]:
!pip install statsmodels

In [8]:
!pip install scikit-learn

In [9]:
# !pip install scikeras[tensorflow]  # For GPU compute platform
!pip install scikeras[tensorflow-cpu]  # For CPU

In [10]:
!pip install yfinance

In [11]:
# !pip install scikeras[tensorflow]  # For GPU compute platform
!pip install scikeras[tensorflow-cpu]  # For CPU

In [17]:
!pip install pyswarms


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 8.8 MB/s eta 0:00:00


In [29]:
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)


In [18]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import mplfinance as mpf

# --- Statsmodels ---
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.ar_model import AutoReg

# --- Scikit-learn ---
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler

# --- TensorFlow / Keras
import tensorflow as tf
from tensorflow import random as tf_random
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    LSTM, Dense, Dropout, Bidirectional, SimpleRNN,
    Conv1D, MaxPooling1D, Flatten, LeakyReLU, Input
)
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adamax, Nadam, Ftrl, Adadelta, Adagrad
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.initializers import GlorotUniform
from pyswarms.single import GlobalBestPSO

# --- Keras Tuner ---
from keras_tuner import RandomSearch, HyperParameters, Objective

# --- Optimization & PSO ---
from hyperopt import Trials, fmin, tpe, hp, STATUS_OK
from pyswarm import pso
from pyswarms.single.global_best import GlobalBestPSO as PSO
from scipy.optimize import minimize
from scipy import stats
from scipy.stats import zscore, randint as sp_randint

# --- Others ---
from math import sqrt
from scikeras.wrappers import KerasRegressor


# ***Get Data***

In [ ]:
start = '2010-01-04'
end = '2020-01-23'


data = yf.download('000001.SS', start, end)


data = data.reset_index()

data = data.dropna()

data


[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-01-04,3289.750000,3295.279053,3243.319092,3243.760010,3243.760010,109400
1,2010-01-05,3254.468018,3290.511963,3221.461914,3282.178955,3282.178955,126200
2,2010-01-06,3277.517090,3295.867920,3253.043945,3254.215088,3254.215088,123600
3,2010-01-07,3253.990967,3268.819092,3176.707031,3192.775879,3192.775879,128600
4,2010-01-08,3177.259033,3198.919922,3149.017090,3195.997070,3195.997070,98400
...,...,...,...,...,...,...,...
2437,2020-01-16,3095.733887,3096.372070,3070.884033,3074.081055,3074.081055,203400
2438,2020-01-17,3081.464111,3091.951904,3067.252930,3075.496094,3075.496094,190300
2439,2020-01-20,3082.113037,3096.311035,3070.479980,3095.787109,3095.787109,210500
2440,2020-01-21,3085.790039,3085.790039,3051.229980,3052.139893,3052.139893,234800


In [ ]:
# Drop the 'Date' column
data = data.drop(columns=['Date'])


In [ ]:
# Determine the length of the training data (70%)
train_len = int(len(data["Adj Close"]) * 0.7)

# Determine the length of the validation data (10%)
val_len = int(len(data["Adj Close"]) * 0.1)

# Set the training, validation, and test data
train_data = data.iloc[:train_len]
val_data = data.iloc[train_len:train_len + val_len]
test_data = data.iloc[train_len + val_len:]


# ***1) Scaling the training data with min-max scaler***

In [ ]:
# Selecting columns
columns = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']


# Calculating Z-Score for each column
z_scores = zscore(train_data[columns])

# Creating a training dataframe without outliers
train_data_without_outliers = train_data[(z_scores < 3).all(axis=1)]


In [ ]:
# Initialize the scaler
scaler = MinMaxScaler()

train_data_scaled = train_data_without_outliers.copy()

# Fit the scaler to the training data and transform
train_data_scaled[columns] = scaler.fit_transform(train_data_without_outliers[columns])

train_data_scaled


,Open,High,Low,Close,Adj Close,Volume
0,0.548970,0.534932,0.559070,0.524307,0.524307,0.131500
1,0.534668,0.533024,0.550302,0.539877,0.539877,0.163150
2,0.544011,0.535168,0.562971,0.528544,0.528544,0.158252
3,0.534475,0.524339,0.532349,0.503645,0.503645,0.167671
4,0.503369,0.496354,0.521241,0.504950,0.504950,0.110776
...,...,...,...,...,...,...
1704,0.495030,0.483586,0.516126,0.480941,0.480941,0.261492
1705,0.485672,0.474754,0.507988,0.473864,0.473864,0.205916
1706,0.478570,0.468967,0.502445,0.471219,0.471219,0.219857
1707,0.473872,0.458809,0.479230,0.467436,0.467436,0.411266


# ***2) Validation data scaling with min-max scaler***

In [ ]:
val_data_scaled = val_data.copy()

val_data_scaled[columns] = scaler.transform(val_data[columns])


val_data_scaled


,Open,High,Low,Close,Adj Close,Volume
1709,0.473983,0.466247,0.501010,0.471320,0.471320,0.173700
1710,0.474066,0.463067,0.499173,0.466573,0.466573,0.158817
1711,0.470356,0.467023,0.499658,0.475424,0.475424,0.155991
1712,0.482356,0.475103,0.511775,0.480950,0.480950,0.175396
1713,0.486073,0.476580,0.514101,0.483292,0.483292,0.162773
...,...,...,...,...,...,...
1948,0.599384,0.588955,0.621458,0.596473,0.596473,0.319329
1949,0.599980,0.587462,0.624185,0.597896,0.597896,0.252826
1950,0.603343,0.591041,0.629136,0.599354,0.599354,0.253391
1951,0.605399,0.593875,0.622850,0.591875,0.591875,0.362472


# ***3) Scaling test data with min-max scaler***

In [ ]:
test_data_scaled = test_data.copy()



test_data_scaled[columns] = scaler.transform(test_data[columns])

test_data_scaled


,Open,High,Low,Close,Adj Close,Volume
1953,0.609300,0.602964,0.634162,0.605728,0.605728,0.417106
1954,0.613883,0.607505,0.641495,0.617920,0.617920,0.339864
1955,0.626749,0.616268,0.651725,0.623233,0.623233,0.391673
1956,0.624874,0.618251,0.652279,0.628703,0.628703,0.335154
1957,0.635961,0.635801,0.663781,0.646998,0.646998,0.375094
...,...,...,...,...,...,...
2437,0.470321,0.455298,0.489898,0.455542,0.455542,0.308591
2438,0.464537,0.453528,0.488441,0.456116,0.456116,0.283911
2439,0.464800,0.455273,0.489736,0.464339,0.464339,0.321967
2440,0.466290,0.451061,0.482014,0.446651,0.446651,0.367747


In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / (y_true + 1e-10))) * 100


In [ ]:
column= ['Open', 'High', 'Low', 'Adj Close', 'Volume']

ARMA-CNN-LSTM

In [31]:

# --- Create dataset with look-back ---
def create_dataset(dataset, look_back=5):
    dataX, dataY = [], []
    dataset = dataset.values # Convert the DataFrame to a numpy array
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i:(i + look_back), :]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 4])   # Use column index 4 for 'Adj Close'
    return np.array(dataX), np.array(dataY)

look_back = 5
columns = train_data.columns

# Scaled datasets
trainX, trainY = create_dataset(train_data_scaled, look_back)
valX, valY = create_dataset(val_data_scaled, look_back)
testX, testY = create_dataset(test_data_scaled, look_back)

trainX_CNN = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], trainX.shape[2]))
valX_CNN = np.reshape(valX, (valX.shape[0], valX.shape[1], valX.shape[2]))
testX_CNN = np.reshape(testX, (testX.shape[0], testX.shape[1], testX.shape[2]))

# --- Train ARMA model on unscaled 'Adj Close' column ---
train_series = train_data['Adj Close'].values
val_series = val_data['Adj Close'].values
test_series = test_data['Adj Close'].values

model_arma = ARIMA(train_series, order=(2, 1, 1))
arma_result = model_arma.fit()

predictions_ARMA_train = arma_result.predict(start=0, end=len(train_series) - 1)
predictions_ARMA_val = arma_result.predict(start=len(train_series), end=len(train_series) + len(val_series) - 1)
predictions_ARMA_test = arma_result.predict(start=len(train_series) + len(val_series),
                                            end=len(train_series) + len(val_series) + len(test_series) - 1)

predictions_ARMA_train = np.array(predictions_ARMA_train).reshape(-1, 1)
predictions_ARMA_val = np.array(predictions_ARMA_val).reshape(-1, 1)
predictions_ARMA_test = np.array(predictions_ARMA_test).reshape(-1, 1)

# --- Define CNN-LSTM model ---
model = Sequential([
    Conv1D(filters=3, kernel_size=2, activation='relu', input_shape=(look_back, len(columns))),
    Conv1D(filters=3, kernel_size=2, activation='relu'),
    Conv1D(filters=3, kernel_size=2, activation='relu'),
    MaxPooling1D(pool_size=2),
    LSTM(72, activation='relu', return_sequences=True),
    LSTM(72, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='relu'),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
history = model.fit(trainX_CNN, trainY, validation_data=(valX_CNN, valY), epochs=200, batch_size=32, verbose=2)

train_predictions_CNN_LSTM = model.predict(trainX_CNN)
val_predictions_CNN_LSTM = model.predict(valX_CNN)
test_predictions_CNN_LSTM = model.predict(testX_CNN)

trainY = trainY.reshape(-1, 1)

# --- Ensure equal lengths for proper ensembling ---
min_len = min(len(train_predictions_CNN_LSTM), len(predictions_ARMA_train), len(trainY))

train_predictions_CNN_LSTM = train_predictions_CNN_LSTM.flatten()[:min_len]
predictions_ARMA_train = predictions_ARMA_train.flatten()[:min_len]
trainY = trainY.flatten()[:min_len]

# --- Define vectorized ensemble loss for PSO ---
def ensemble_loss(weights):
    w1 = weights[:, 0]
    w2 = weights[:, 1]

    pred_cnn = np.expand_dims(train_predictions_CNN_LSTM, axis=0)
    pred_arma = np.expand_dims(predictions_ARMA_train, axis=0)
    true_vals = np.expand_dims(trainY, axis=0)

    combined_pred = w1[:, None] * pred_cnn + w2[:, None] * pred_arma
    mse = np.mean((true_vals - combined_pred) ** 2, axis=1)
    return mse

# --- Optimize weights using PSO ---
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
bounds = (np.array([0, 0]), np.array([1, 1]))
optimizer = GlobalBestPSO(n_particles=30, dimensions=2, options=options, bounds=bounds)

cost, pos = optimizer.optimize(ensemble_loss, iters=100)
w1, w2 = pos

# --- Combined predictions using optimized weights ---
train_predictions_combined = w1 * train_predictions_CNN_LSTM + w2 * predictions_ARMA_train

# --- Validation ensemble ---
min_len_val = min(len(val_predictions_CNN_LSTM), len(predictions_ARMA_val), len(valY))
val_predictions_combined = w1 * val_predictions_CNN_LSTM.flatten()[:min_len_val] + \
                           w2 * predictions_ARMA_val.flatten()[:min_len_val]
valY = valY.flatten()[:min_len_val]

# --- Test ensemble ---
min_len_test = min(len(test_predictions_CNN_LSTM), len(predictions_ARMA_test), len(testY))
test_predictions_combined = w1 * test_predictions_CNN_LSTM.flatten()[:min_len_test] + \
                            w2 * predictions_ARMA_test.flatten()[:min_len_test]
testY = testY.flatten()[:min_len_test]


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
52/52 - 10s - 192ms/step - loss: 0.1032 - val_loss: 0.2333
Epoch 2/200
52/52 - 1s - 12ms/step - loss: 0.0810 - val_loss: 0.1916
Epoch 3/200
52/52 - 1s - 22ms/step - loss: 0.0653 - val_loss: 0.1586
Epoch 4/200
52/52 - 1s - 10ms/step - loss: 0.0547 - val_loss: 0.1329
Epoch 5/200
52/52 - 1s - 14ms/step - loss: 0.0479 - val_loss: 0.1135
Epoch 6/200
52/52 - 1s - 11ms/step - loss: 0.0437 - val_loss: 0.0990
Epoch 7/200
52/52 - 0s - 6ms/step - loss: 0.0413 - val_loss: 0.0884
Epoch 8/200
52/52 - 0s - 6ms/step - loss: 0.0401 - val_loss: 0.0810
Epoch 9/200
52/52 - 0s - 6ms/step - loss: 0.0394 - val_loss: 0.0757
Epoch 10/200
52/52 - 0s - 6ms/step - loss: 0.0391 - val_loss: 0.0722
Epoch 11/200
52/52 - 0s - 5ms/step - loss: 0.0390 - val_loss: 0.0699
Epoch 12/200
52/52 - 0s - 6ms/step - loss: 0.0389 - val_loss: 0.0684
Epoch 13/200
52/52 - 1s - 12ms/step - loss: 0.0389 - val_loss: 0.0674
Epoch 14/200
52/52 - 0s - 6ms/step - loss: 0.0389 - val_loss: 0.0668
Epoch 15/200
52/52 - 0s - 6ms/step

2025-06-24 00:31:54,666 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|100/100, best_cost=0.653
2025-06-24 00:31:54,878 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.6529915735377392, best pos: [0.1352242  0.00038826]


In [33]:
# --- Inverse scaling function ---
def inverse_transform(scaled_preds, scaler, columns, original_len):
    reshaped_preds = scaled_preds.reshape(-1, 1)
    extended = np.c_[reshaped_preds, np.zeros((reshaped_preds.shape[0], len(columns) - 1))]
    inversed = scaler.inverse_transform(extended)[:, 4]
    return inversed[:original_len]

# --- Inverse transform for predictions and actual values ---
train_predictions_original = inverse_transform(train_predictions_combined, scaler, columns, len(trainY))
val_predictions_original = inverse_transform(val_predictions_combined, scaler, columns, len(valY))
test_predictions_original = inverse_transform(test_predictions_combined, scaler, columns, len(testY))

trainY_original = inverse_transform(trainY, scaler, columns, len(trainY))
valY_original = inverse_transform(valY, scaler, columns, len(valY))
testY_original = inverse_transform(testY, scaler, columns, len(testY))

# --- Function to compute and print evaluation metrics ---
def print_metrics(true_scaled, pred_scaled, true_original, pred_original, dataset_name):
    rmse = np.sqrt(mean_squared_error(true_original, pred_original))
    r2 = r2_score(true_scaled, pred_scaled)
    mape = mean_absolute_percentage_error(true_scaled, pred_scaled)
    mae = mean_absolute_error(true_scaled, pred_scaled)

    print(f"{dataset_name} RMSE: {rmse:.5f}")
    print(f"{dataset_name} R^2: {r2:.5f}")
    print(f"{dataset_name} MAPE: {mape:.5f}")
    print(f"{dataset_name} MAE: {mae:.5f}")
    print('-------------------------------------------')

# --- Print metrics for each dataset ---
print_metrics(trainY, train_predictions_combined, trainY_original, train_predictions_original, "Train")
print_metrics(valY, val_predictions_combined, valY_original, val_predictions_original, "Validation")
print_metrics(testY, test_predictions_combined, testY_original, test_predictions_original, "Test")


Train RMSE: 142.52831
Train R^2: 0.97315
Train MAPE: 0.00982
Train MAE: 1.25379
-------------------------------------------
Validation RMSE: 176.89320
Validation R^2: 0.95248
Validation MAPE: 0.01234
Validation MAE: 1.49562
-------------------------------------------
Test RMSE: 163.27485
Test R^2: 0.96512
Test MAPE: 0.01111
Test MAE: 1.38274
-------------------------------------------


ARMA-CNN-BiLSTM

In [34]:
# --- 1. Define dataset creation function ---
def create_dataset(dataset, look_back=5):
    dataX, dataY = [], []
    dataset = dataset.values  # Convert DataFrame to numpy array
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i:(i + look_back), :]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 4])  # Use column index 4 for 'Adj Close'
    return np.array(dataX), np.array(dataY)

# --- 2. Create and scale data (train_data_scaled, val_data_scaled, test_data_scaled) ---
look_back = 5
columns = train_data_scaled.columns

trainX, trainY = create_dataset(train_data_scaled, look_back)
valX, valY = create_dataset(val_data_scaled, look_back)
testX, testY = create_dataset(test_data_scaled, look_back)

trainX_CNN = trainX.reshape(trainX.shape[0], trainX.shape[1], trainX.shape[2])
valX_CNN = valX.reshape(valX.shape[0], valX.shape[1], valX.shape[2])
testX_CNN = testX.reshape(testX.shape[0], testX.shape[1], testX.shape[2])

# --- 3. Define CNN-BiLSTM model ---
model = Sequential()
model.add(Conv1D(filters=3, kernel_size=2, activation='relu', input_shape=(look_back, len(columns))))
model.add(Conv1D(filters=3, kernel_size=2, activation='relu'))
model.add(Conv1D(filters=3, kernel_size=2, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Bidirectional(LSTM(72, activation='relu', return_sequences=True)))
model.add(Bidirectional(LSTM(72, activation='relu')))
model.add(Dropout(0.5))
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1))

model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# --- 4. Train the model ---
history = model.fit(trainX_CNN, trainY, validation_data=(valX_CNN, valY), epochs=200, batch_size=32, verbose=0)

# --- 5. Make predictions with the CNN-BiLSTM model ---
train_pred_cnn = model.predict(trainX_CNN).flatten()
val_pred_cnn = model.predict(valX_CNN).flatten()
test_pred_cnn = model.predict(testX_CNN).flatten()

# --- 6. Fit ARMA model on trainY ---
arma_order = (2, 1, 1)  # (p,d,q) for ARIMA
trainY_series = pd.Series(trainY)

arma_model = ARIMA(trainY_series, order=arma_order)
arma_result = arma_model.fit()

# Predict with ARMA on train set
train_pred_arma = arma_result.predict(start=0, end=len(trainY)-1).values

# Since we don't have ARMA predictions for val and test sets, we only use CNN predictions for those

# --- 7. Define PSO objective function (vectorized for multiple particles) ---
def objective(weights):
    epsilon = 1e-10
    w1 = weights[:, 0]
    w2 = weights[:, 1]

    # Expand dimensions for broadcasting
    train_pred_cnn_exp = np.expand_dims(train_pred_cnn, axis=0)
    train_pred_arma_exp = np.expand_dims(train_pred_arma, axis=0)

    val_pred_cnn_exp = np.expand_dims(val_pred_cnn, axis=0)
    test_pred_cnn_exp = np.expand_dims(test_pred_cnn, axis=0)

    trainY_exp = np.expand_dims(trainY, axis=0)
    valY_exp = np.expand_dims(valY, axis=0)
    testY_exp = np.expand_dims(testY, axis=0)

    # Compute ensemble predictions for each particle
    train_pred = (w1[:, None] * train_pred_cnn_exp + w2[:, None] * train_pred_arma_exp) / (w1[:, None] + w2[:, None] + epsilon)
    val_pred = val_pred_cnn_exp  # Only CNN, since ARMA is not available
    test_pred = test_pred_cnn_exp

    # Compute RMSE for each particle
    train_rmse = np.sqrt(np.mean((trainY_exp - train_pred) ** 2, axis=1))
    val_rmse = np.sqrt(np.mean((valY_exp - val_pred) ** 2, axis=1))
    test_rmse = np.sqrt(np.mean((testY_exp - test_pred) ** 2, axis=1))

    return train_rmse + val_rmse + test_rmse

# --- 8. PSO settings ---
lb = [0, 0]
ub = [1, 1]
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}

pso = PSO(n_particles=50, dimensions=2, options=options, bounds=(lb, ub))

cost, pos = pso.optimize(objective, iters=100)

print(f"Optimal weights: {pos}")
print(f"Minimum objective value: {cost}")

# --- 9. Final predictions using optimized weights ---
w1_opt, w2_opt = pos
epsilon = 1e-10
train_pred_final = (w1_opt * train_pred_cnn + w2_opt * train_pred_arma) / (w1_opt + w2_opt + epsilon)
val_pred_final = val_pred_cnn  # No ARMA for val
test_pred_final = test_pred_cnn  # No ARMA for test


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


2025-06-24 00:36:16,311 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|100/100, best_cost=0.114
2025-06-24 00:36:16,611 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.11410379711670982, best pos: [0.07022584 0.39490146]


Optimal weights: [0.07022584 0.39490146]
Minimum objective value: 0.11410379711670982


In [36]:
# --- Inverse scaling function ---
def inverse_transform(scaled_preds, scaler, columns, original_len):
    reshaped_preds = scaled_preds.reshape(-1, 1)
    extended = np.c_[reshaped_preds, np.zeros((reshaped_preds.shape[0], len(columns) - 1))]
    inversed = scaler.inverse_transform(extended)[:, 4]
    return inversed[:original_len]

# --- Convert predictions and true values back to original scale ---
train_pred_original = inverse_transform(train_pred_final, scaler, columns, len(train_pred_final))
val_pred_original = inverse_transform(val_pred_final, scaler, columns, len(val_pred_final))
test_pred_original = inverse_transform(test_pred_final, scaler, columns, len(test_pred_final))

trainY_original = inverse_transform(trainY, scaler, columns, len(trainY))
valY_original = inverse_transform(valY, scaler, columns, len(valY))
testY_original = inverse_transform(testY, scaler, columns, len(testY))

# --- Compute and print evaluation metrics ---
def print_metrics(true_scaled, pred_scaled, true_original, pred_original, dataset_name):
    rmse = np.sqrt(mean_squared_error(true_original, pred_original))
    r2 = r2_score(true_scaled, pred_scaled)
    mape = mean_absolute_percentage_error(true_scaled, pred_scaled)
    mae = mean_absolute_error(true_scaled, pred_scaled)

    print(f"{dataset_name} RMSE: {rmse:.5f}")
    print(f"{dataset_name} R^2: {r2:.5f}")
    print(f"{dataset_name} MAPE: {mape:.5f}")
    print(f"{dataset_name} MAE: {mae:.5f}")
    print('-------------------------------------------')

# --- Print results ---
print_metrics(trainY, train_pred_final, trainY_original, train_pred_original, "Train")
print_metrics(valY, val_pred_final, valY_original, val_pred_original, "Validation")
print_metrics(testY, test_pred_final, testY_original, test_pred_original, "Test")


Train RMSE: 44.15548
Train R^2: 0.99161
Train MAPE: 0.01172
Train MAE: 31.13293
-------------------------------------------
Validation RMSE: 25.60751
Validation R^2: 0.93201
Validation MAPE: 0.00659
Validation MAE: 21.22402
-------------------------------------------
Test RMSE: 36.54943
Test R^2: 0.97262
Test MAPE: 0.00911
Test MAE: 26.55602
-------------------------------------------
